In [48]:
import os
import string
import annoy

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from pymorphy2 import MorphAnalyzer
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize, RegexpTokenizer

import numpy as np
from tqdm import tqdm_notebook
import pandas as pd

import tqdm
from string import punctuation
import re

[nltk_data] Downloading package stopwords to /home/nlp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Предобработаем ответы mail.ru из файла: к каждому вопросу присоединим 1 ответ и запишем в файл на будущее. Это позволит нам сэкономить время и ресурсы при дальнейшем препроцессинге текста

In [49]:
question = None
written = False

'''Идем по всем записям, берем первую строку как вопрос и после знака --- находим ответ'''
with open('prepared_answers.txt', 'w') as fout:
    with open('answers.txt', 'r') as fin:
        for line in tqdm.notebook.tqdm(fin):
            if line.startswith('---'):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace('\t', ' ').strip() + '\t' + line.replace('\t', ' '))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

/tmp/ipykernel_629308/1279344030.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

Предобработаем текст, а дальше обучим word2vec и получим эмбеддинги

In [54]:
def preprocess_txt(txt: str, sw = []) -> str:
    exclude = list(punctuation)
    '''Стоп-слова из библиотеки nltk'''
    sw_1 = list(stopwords.words('russian'))
    morpher = MorphAnalyzer()
    txt = str(txt)
    '''Делим предложение на слова по знакам препинания, приводим все слова к нижнему регистру, удаляем знаки препинания, стоп-слова и цифры'''
    tokenizer = RegexpTokenizer("\w+|[^\w\s]+")
    txt = [i.lower() for i in tokenizer.tokenize(txt) if i not in exclude and i not in sw and i not in sw_1 and len(i) > 2 and \
          re.search('(\d+)|\)|\(|\.|\:|\;|\!|\?|\<|\>|[A-Za-z]', i) is None]
    '''Делаем лемматизацию'''
    txt = [morpher.parse(word)[0].normal_form for word in txt]
    return " ".join(txt)

In [57]:
sentences = []
c = 0

with open('answers.txt', 'r') as fin:
    for line in tqdm.notebook.tqdm(fin):
        spls = preprocess_txt(line, sw)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

0it [00:00, ?it/s]

In [58]:
# Обучим модель word2vec на наших вопросах
sentences = [i for i in sentences if len(i) > 2]
model = Word2Vec(sentences=sentences, vector_size=100, min_count=1, window=5)
#model.save("w2v_model")

In [61]:
import pickle

In [62]:
def save_model(model, name):
    return pickle.dump(model, open(f'models/{name}.sav', 'wb'))

In [63]:
save_model(model, 'w2v_model')

Сложим в индекс все вопросы. Используем библиотеку annoy. Проходимся по всем ответам, считаем, что вектор предложения - сумма word2vecов слов (усредненная), которые входят в него

In [66]:
index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with open('prepared_answers.txt', 'r') as f:
    for line in tqdm.notebook.tqdm(f):
        n_w2v = 0
        spls = line.split('\t')
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0], sw)
        vector = np.zeros(100)
        for word in question:
            if word in model.wv:
                vector += model.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        index.add_item(counter, vector)

index.build(10)
#index.save('speaker.ann')

0it [00:00, ?it/s]

Реализуем метод, который получает на вход вопрос и находит ответ к нему.
Препроцессим вопрос, находим ближайший вопрос и выбираем ответ на ближайший вопрос.

In [18]:
def find_answer(question):
    preprocessed_question = preprocess_text(question)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_question:
        if word in model.wv:
            vector += model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    answer_index = index.get_nns_by_vector(vector, 1)
    return index_map[answer_index[0]]

In [43]:
find_answer('медное литье')

'Про свою жену я этого сказать не могу, ведь она меня любит.. \n'

In [ ]:
sw = ['всё', 'тому', 'представьте', 'чьё', 'смысле', 'статься', 'знаешь', 'получается', 'или', 'выяснилось', 'удивительно',
      'которых', 'оти', 'значится', 'сказано', 'никогда', 'об', 'образом', 'моему', 'самом', 'свыше', 'во-вторых', 'как', 'чья', 'иметь', 
      'следовательно', 'серьезно', 'точки', 'точнее', 'поди', 'бы', 'но', 'само', 'осле', 'конено', 'около', 'из-за', 'отому', 'скажем',
      'новость', 'достаточно', 'издревле', 'когда', 'любом', 'ваша', 'мол', 'конец', 'словно', 'никто', 'уж', 'да', 'актуальный', 'попросту',
      'бесспорно', 'долженствовать', 'было', 'которая', 'хорошо', 'случайно', 'хоть', 'ко', 'видать', 'повседневной', 'этот', 'позволь',
      'наверно', 'прочему', 'самым', 'буквально', 'здесь' 'воистину', 'право', 'ничего', 'понятно', 'разнообразный', 'вообразите', 'июль',
      'эта', 'тогда', 'дальше', 'конечно', 'наоборот', 'бывает', 'твоя', 'мне', 'полагать', 'важно', 'нам', 'представленный', 'проще',
      'очередной', 'выясняется', 'странно', 'устоявшееся', 'моя', 'всевозможный', 'долженствующий', 'прочий', 'где', 'откровенно', 'принято',
      'беспрецедентный', 'целом', 'один', 'нашей', 'абсолютно', 'следствие', 'также', 'они', 'после', 'согласиться', 'реальный', 'по-твоему',
      'имеется', 'лет', 'небось', 'казалось', 'своему', 'скорее', 'неизгладимый', 'напротив', 'март', 'наверняка', 'обычно', 'нынче',
      'моей', 'примечательно', 'против', 'наконец', 'так', 'тыс', 'какой', 'много', 'ней', 'если', 'должно', 'ваш', 'нее', 'быть', 'этом',
      'чем', 'возможно', 'вестимо', 'знать', 'вроем', 'выражаясь', 'одним', 'теерь', 'кажется', 'многие', 'от', 'есть', 'себе', 'просто',
      'всяком', 'даже', 'ничто', 'обычаю', 'знает', 'исходя', 'несколько', 'предельно', 'ли', 'она', 'ерез', 'вас', 'ведь', 'имеются', 'более',
      'он', 'кто', 'по', 'уть', 'свою' 'твоему', 'них', 'кстати', 'од', 'крайне', 'ри', 'говори', 'примеру', 'их', 'вашему', 'всякого',
      'исключение', 'через', 'того', 'год', 'оказывается', 'по-ихнему', 'вишь', 'конце', 'вашего', 'тоб', 'кб', 'пожалуйста', 'сейас',
      'вам', 'эй', 'над', 'естественно', 'без', 'нибудь', 'нечего', 'еред', 'куда', 'вообще-то', 'менее', 'по-вашему', 'хм', 'что', 'про',
      'всей', 'им', 'наша', 'кажись', 'твоего', 'своей', 'наверное', 'собой', 'нами', 'предположительно', 'мы', 'всех', 'по-моему', 'исстари',
      'себя', 'некто', 'иногда', 'совершенно', 'для', 'вы', 'этих', 'как-то', 'разумеется', 'ему', 'всегда', 'чего', 'другие', 'во-первых',
      'позвольте', 'вип', 'очевидно', 'так-то', 'замыслу', 'нарочно', 'короче', 'то', 'помилуйте', 'сути', 'по-хорошему', 'иначе', 'нередко',
      'чтобы', 'всего', 'все', 'надо', 'весь', 'раз', 'известно', 'из', 'свой', 'по-видимому', 'тут', 'под', 'мой', 'ул', 'луше', 'ей', 'только',
      'вероятно', 'видимо', 'руб', 'слову', 'нет', 'за', 'тем', 'во', 'например', 'помимо', 'такой', 'будет', 'помилуй', 'тоже', 'скажут',
      'видно', 'пожалуй', 'общеизвестно', 'чье', 'вдобавок', 'впрочем', 'соответственно', 'зачастую', 'сам', 'том', 'там', 'полагается',
      'меня', 'однако', 'слышь', 'итак', 'подобное', 'действительно', 'еще', 'его', 'наше', 'данный', 'твой', 'ай', 'необходимо', 'три',
      'далее', 'тот', 'эм', 'же', 'жаль', 'другими', 'видишь', 'некоторых', 'прежде', 'который', 'допустим', 'положено', 'по-нашему', 'весьма',
      'не', 'ро', 'той', 'этого', 'эти', 'напомним', 'правда', 'выходит', 'перед', 'определенно', 'какая', 'относительно', 'сразу', 'имя',
      'чей', 'ее', 'отом', 'неё', 'это', 'два', 'эту', 'разве', 'вне', 'тобы', 'знамо', 'всем', 'вернее', 'будто', 'вновь', 'которые', 'данным',
      'кроме', 'таким', 'нечто', 'кого', 'больше', 'ом', 'уже', 'прочего', 'очень', 'при', 'те', 'вообще', 'ясно', 'ну', 'октябрь', 'до',
      'на', 'в-третьих', 'ежели', 'общем', 'ещё', 'похоже', 'ибо', 'оно', 'ты', 'сверх', 'значит', 'безусловно', 'правильнее', 'ниего', 'него',      'совсем',
      'вами', 'общем-то', 'всю', 'наш', 'всему', 'всякий', 'ой', 'этой', 'ни', 'со', 'вот', 'день', 'может', 'мб', 'тебя', 'нас',
      'оять', 'хотя', 'прочим', 'этим', 'между', 'ним', 'вероятнее', 'бывало', 'примерно', 'вдруг', 'либо']